# Register model and deploy locally

See: https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/deployment/deploy-to-local/register-model-deploy-local.ipynb

- Register model
- Deploy the image as a web service in a local Docker container.
- Quickly test changes to your entry script by reloading the local service.
- Optionally, you can also make changes to model, conda or extra_docker_file_steps and update local service

## Assumptions

- Data is already preprocessed in `data/`

# Setup Workspace and Prereqs

In [10]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.50.0


In [2]:
from azureml.core.workspace import Workspace

# the config file is one directory up
# NOTE: this is an interactive authorization!
ws = Workspace.from_config('..')
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

Nanostics_ML_Workspace
SummerProjects2023
canadacentral
1b1ae7cf-df24-428b-8bb9-e4dd07869ac9


# Train Model

the code is ripped from `../train.ipynb`, just without the MLFlow stuff

In [80]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd 

def split_and_scale(df):
    y = df["diagnosis_01"]
    X = df.drop(["diagnosis_01"],axis=1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, X.columns

In [81]:
from sklearn.svm import SVC

def train_svc(df):
    X_train, X_test, y_train, y_test, train_cols = split_and_scale(df)
    svc = SVC()
    svc.fit(X_train, y_train)
    return svc, X_test, y_test, train_cols

In [86]:
import pandas as pd 


data = '../data/cleaned-wisconsin-lof.parquet'
df = pd.read_parquet(data)

svc, X_test, y_test, train_cols = train_svc(df)

In [87]:
# df_test = pd.DataFrame(X_test, columns=df.columns.drop('diagnosis_01'))

# d = df_test.iloc[0].to_dict()
# # # pd.DataFrame(d, index=[0])
# # svc.predict(pd.DataFrame(d, index=[0]))
X_test[0]

array([-1.53587537, -1.59868622, -1.2644576 , -1.07047828, -1.01543024,
       -0.73519693, -0.39527791, -0.28990047,  0.45715098, -0.39584766,
       -0.36948983,  1.08027519, -0.87830952, -0.86517173, -0.65332827,
        0.93093931,  0.32816649, -1.53899135, -0.76896925, -0.66501272,
       -1.68158125, -1.27075786, -1.30537918, -1.38663635, -0.99976219,
       -0.79221752])

In [88]:
import joblib

joblib.dump(svc, 'model.pkl')

['model.pkl']

## Register Model

In [89]:
from azureml.core.model import Model

model = Model.register(model_path="model.pkl",
                       model_name="wisconsin-BCa-model",
                       tags={'area': "cancer", 'type': "svc"},
                       description="SVC model to predict Breast Cancer",
                       workspace=ws)

Registering model wisconsin-BCa-model


## Create Environment + Inference Config

straight copy-paste

In [90]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [91]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=environment)

## Deploy Model as a Docker Service

This is the good stuff!!!!

In [92]:
from azureml.core.webservice import LocalWebservice

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

/tmp/ipykernel_14839/1718659047.py:6: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  local_service = Model.deploy(ws, "test", [model], inference_config, deployment_config)
Failed to create Docker client. Is Docker running/installed?
When you deploy locally, we download a dockerfile
execute docker build on it, and docker run the built container for you
Error: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))



WebserviceException: WebserviceException:
	Message: Failed to create Docker client. Is Docker running/installed?
When you deploy locally, we download a dockerfile
execute docker build on it, and docker run the built container for you
Error: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Failed to create Docker client. Is Docker running/installed?\nWhen you deploy locally, we download a dockerfile\nexecute docker build on it, and docker run the built container for you\nError: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))"
    }
}